In [1]:
import firebase_admin
from firebase_admin import db,credentials
cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred, {'databaseURL': 'https://fir-demo-29d5b-default-rtdb.firebaseio.com'})

In [2]:
ref = db.reference('/')
data = ref.child('songs').order_by_child('id').get()
values = []
for key in data:
    values.append(data[key])
values

[{'320kbps': 'false',
  'album': 'Miền Tây Official',
  'album_id': 'UCQuiXXMBJH-HpYffyJ5jvdg',
  'artist': 'Miền Tây Official',
  'duration': '245',
  'genre': 'YouTube',
  'has_lyrics': 'false',
  'id': '-4eFAzI-7o0',
  'image': 'https://img.youtube.com/vi/-4eFAzI-7o0/maxresdefault.jpg',
  'language': 'YouTube',
  'perma_url': 'https://www.youtube.com/watch?v=-4eFAzI-7o0',
  'release_date': '2022-11-19 00:00:00.000',
  'subtitle': 'Miền Tây Official',
  'title': 'THƯƠNG THÌ VỀ ĐÂY | TIẾN TỚI | Official Music Video | Mình về Đồng Tháp nghe câu hò hò ơi',
  'url': 'https://rr7---sn-8qj-nboey.googlevideo.com/videoplayback?expire=1686071122&ei=8hJ_ZJ_FF-fb2roP8bS3-AM&ip=113.185.84.90&id=o-AD1lNpbg9XCk0ZaH97wmnNUZK8O6nwronCQBxOvUEGXG&itag=251&source=youtube&requiressl=yes&mh=Pp&mm=31%2C29&mn=sn-8qj-nboey%2Csn-8qj-i5o66&ms=au%2Crdu&mv=m&mvi=7&pl=24&initcwndbps=561250&spc=qEK7B-tdk6xkFve9eUrU9rGeT6-625o&vprv=1&svpuc=1&mime=audio%2Fwebm&gir=yes&clen=4394107&dur=244.901&lmt=1669188913136787&m

In [3]:
import pandas as pd

song = pd.DataFrame(values)
song = song.rename(columns={'id' : 'songId'})
song.head()

,320kbps,album,album_id,artist,duration,genre,has_lyrics,songId,image,language,perma_url,release_date,subtitle,title,url,year
0,false,Miền Tây Official,UCQuiXXMBJH-HpYffyJ5jvdg,Miền Tây Official,245,YouTube,false,-4eFAzI-7o0,https://img.youtube.com/vi/-4eFAzI-7o0/maxresd...,YouTube,https://www.youtube.com/watch?v=-4eFAzI-7o0,2022-11-19 00:00:00.000,Miền Tây Official,THƯƠNG THÌ VỀ ĐÂY | TIẾN TỚI | Official Music ...,https://rr7---sn-8qj-nboey.googlevideo.com/vid...,2022
1,false,PHÚC DU,UCPQM-wqVHzULADG9zJir5HQ,PHÚC DU,263,YouTube,false,-7sISWuTdj0,https://img.youtube.com/vi/-7sISWuTdj0/maxresd...,YouTube,https://www.youtube.com/watch?v=-7sISWuTdj0,2023-02-13 00:00:00.000,PHÚC DU,PHÚC DU - yêu anh đi mẹ anh bán bánh mì (MV OF...,https://rr3---sn-8qj-q5qes.googlevideo.com/vid...,2023
2,false,Sơn Tùng M-TP,UC3muIvzjhubNpJ4Pn_0kCQw,Sơn Tùng M-TP,258,YouTube,false,-E2u6exhzPE,https://img.youtube.com/vi/-E2u6exhzPE/maxresd...,YouTube,https://www.youtube.com/watch?v=-E2u6exhzPE,2023-05-04 00:00:00.000,Sơn Tùng M-TP - Topic,MAKING MY WAY,https://rr3---sn-8qj-nboey.googlevideo.com/vid...,2023
3,false,HYBE LABELS,UC3IZKseVpdzPSBaWxBxundA,HYBE LABELS,213,YouTube,false,-GQg25oP0S4,https://img.youtube.com/vi/-GQg25oP0S4/maxresd...,YouTube,https://www.youtube.com/watch?v=-GQg25oP0S4,2023-04-24 00:00:00.000,HYBE LABELS,SEVENTEEN (세븐틴) '손오공' Official MV,https://rr1---sn-8qj-nbo6y.googlevideo.com/vid...,2023
4,false,1theK (원더케이),UCweOkPb1wVVH0Q0Tlj4a5Pw,1theK (원더케이),226,YouTube,false,-I0Yj7NLxu0,https://img.youtube.com/vi/-I0Yj7NLxu0/maxresd...,YouTube,https://www.youtube.com/watch?v=-I0Yj7NLxu0,2023-05-18 00:00:00.000,1theK (원더케이),[MV] ATBO(에이티비오) _ Next to Me,https://rr2---sn-8qj-nbo66.googlevideo.com/vid...,2023


In [5]:
stat = ref.child('stats').order_by_child('songId').get()
stats = []
for key in stat:
    stats.append(stat[key])
stat_df = pd.DataFrame(stats)
stat_df.head()

,listenCount,songId,userId
0,14,-7sISWuTdj0,f6110910-f7bb-11ed-b8bd-ffe1409aca9d
1,7,-7sISWuTdj0,d9155580-f7c7-11ed-ab2f-f146d31a9014
2,5,-7sISWuTdj0,08ed0940-f7ca-11ed-b7d7-b75e9c825c2a
3,18,-E2u6exhzPE,0947e130-f7cf-11ed-92de-f7e7740e5f43
4,21,-GQg25oP0S4,0c2983d0-f16d-11ed-9c88-514c38a48ebe


In [6]:
song_df = pd.merge(stat_df, song.drop_duplicates(['songId']), on='songId', how='left')
song_df.head()

,listenCount,songId,userId,320kbps,album,album_id,artist,duration,genre,has_lyrics,image,language,perma_url,release_date,subtitle,title,url,year
0,14,-7sISWuTdj0,f6110910-f7bb-11ed-b8bd-ffe1409aca9d,false,PHÚC DU,UCPQM-wqVHzULADG9zJir5HQ,PHÚC DU,263,YouTube,false,https://img.youtube.com/vi/-7sISWuTdj0/maxresd...,YouTube,https://www.youtube.com/watch?v=-7sISWuTdj0,2023-02-13 00:00:00.000,PHÚC DU,PHÚC DU - yêu anh đi mẹ anh bán bánh mì (MV OF...,https://rr3---sn-8qj-q5qes.googlevideo.com/vid...,2023
1,7,-7sISWuTdj0,d9155580-f7c7-11ed-ab2f-f146d31a9014,false,PHÚC DU,UCPQM-wqVHzULADG9zJir5HQ,PHÚC DU,263,YouTube,false,https://img.youtube.com/vi/-7sISWuTdj0/maxresd...,YouTube,https://www.youtube.com/watch?v=-7sISWuTdj0,2023-02-13 00:00:00.000,PHÚC DU,PHÚC DU - yêu anh đi mẹ anh bán bánh mì (MV OF...,https://rr3---sn-8qj-q5qes.googlevideo.com/vid...,2023
2,5,-7sISWuTdj0,08ed0940-f7ca-11ed-b7d7-b75e9c825c2a,false,PHÚC DU,UCPQM-wqVHzULADG9zJir5HQ,PHÚC DU,263,YouTube,false,https://img.youtube.com/vi/-7sISWuTdj0/maxresd...,YouTube,https://www.youtube.com/watch?v=-7sISWuTdj0,2023-02-13 00:00:00.000,PHÚC DU,PHÚC DU - yêu anh đi mẹ anh bán bánh mì (MV OF...,https://rr3---sn-8qj-q5qes.googlevideo.com/vid...,2023
3,18,-E2u6exhzPE,0947e130-f7cf-11ed-92de-f7e7740e5f43,false,Sơn Tùng M-TP,UC3muIvzjhubNpJ4Pn_0kCQw,Sơn Tùng M-TP,258,YouTube,false,https://img.youtube.com/vi/-E2u6exhzPE/maxresd...,YouTube,https://www.youtube.com/watch?v=-E2u6exhzPE,2023-05-04 00:00:00.000,Sơn Tùng M-TP - Topic,MAKING MY WAY,https://rr3---sn-8qj-nboey.googlevideo.com/vid...,2023
4,21,-GQg25oP0S4,0c2983d0-f16d-11ed-9c88-514c38a48ebe,false,HYBE LABELS,UC3IZKseVpdzPSBaWxBxundA,HYBE LABELS,213,YouTube,false,https://img.youtube.com/vi/-GQg25oP0S4/maxresd...,YouTube,https://www.youtube.com/watch?v=-GQg25oP0S4,2023-04-24 00:00:00.000,HYBE LABELS,SEVENTEEN (세븐틴) '손오공' Official MV,https://rr1---sn-8qj-nbo6y.googlevideo.com/vid...,2023


In [7]:
song_df['song'] = song_df['title']+' - '+song_df['artist']
song_df.head()

,listenCount,songId,userId,320kbps,album,album_id,artist,duration,genre,has_lyrics,image,language,perma_url,release_date,subtitle,title,url,year,song
0,14,-7sISWuTdj0,f6110910-f7bb-11ed-b8bd-ffe1409aca9d,false,PHÚC DU,UCPQM-wqVHzULADG9zJir5HQ,PHÚC DU,263,YouTube,false,https://img.youtube.com/vi/-7sISWuTdj0/maxresd...,YouTube,https://www.youtube.com/watch?v=-7sISWuTdj0,2023-02-13 00:00:00.000,PHÚC DU,PHÚC DU - yêu anh đi mẹ anh bán bánh mì (MV OF...,https://rr3---sn-8qj-q5qes.googlevideo.com/vid...,2023,PHÚC DU - yêu anh đi mẹ anh bán bánh mì (MV OF...
1,7,-7sISWuTdj0,d9155580-f7c7-11ed-ab2f-f146d31a9014,false,PHÚC DU,UCPQM-wqVHzULADG9zJir5HQ,PHÚC DU,263,YouTube,false,https://img.youtube.com/vi/-7sISWuTdj0/maxresd...,YouTube,https://www.youtube.com/watch?v=-7sISWuTdj0,2023-02-13 00:00:00.000,PHÚC DU,PHÚC DU - yêu anh đi mẹ anh bán bánh mì (MV OF...,https://rr3---sn-8qj-q5qes.googlevideo.com/vid...,2023,PHÚC DU - yêu anh đi mẹ anh bán bánh mì (MV OF...
2,5,-7sISWuTdj0,08ed0940-f7ca-11ed-b7d7-b75e9c825c2a,false,PHÚC DU,UCPQM-wqVHzULADG9zJir5HQ,PHÚC DU,263,YouTube,false,https://img.youtube.com/vi/-7sISWuTdj0/maxresd...,YouTube,https://www.youtube.com/watch?v=-7sISWuTdj0,2023-02-13 00:00:00.000,PHÚC DU,PHÚC DU - yêu anh đi mẹ anh bán bánh mì (MV OF...,https://rr3---sn-8qj-q5qes.googlevideo.com/vid...,2023,PHÚC DU - yêu anh đi mẹ anh bán bánh mì (MV OF...
3,18,-E2u6exhzPE,0947e130-f7cf-11ed-92de-f7e7740e5f43,false,Sơn Tùng M-TP,UC3muIvzjhubNpJ4Pn_0kCQw,Sơn Tùng M-TP,258,YouTube,false,https://img.youtube.com/vi/-E2u6exhzPE/maxresd...,YouTube,https://www.youtube.com/watch?v=-E2u6exhzPE,2023-05-04 00:00:00.000,Sơn Tùng M-TP - Topic,MAKING MY WAY,https://rr3---sn-8qj-nboey.googlevideo.com/vid...,2023,MAKING MY WAY - Sơn Tùng M-TP
4,21,-GQg25oP0S4,0c2983d0-f16d-11ed-9c88-514c38a48ebe,false,HYBE LABELS,UC3IZKseVpdzPSBaWxBxundA,HYBE LABELS,213,YouTube,false,https://img.youtube.com/vi/-GQg25oP0S4/maxresd...,YouTube,https://www.youtube.com/watch?v=-GQg25oP0S4,2023-04-24 00:00:00.000,HYBE LABELS,SEVENTEEN (세븐틴) '손오공' Official MV,https://rr1---sn-8qj-nbo6y.googlevideo.com/vid...,2023,SEVENTEEN (세븐틴) '손오공' Official MV - HYBE LABELS


In [39]:
song_grouped = song_df.groupby(['songId']).agg({'listenCount': ['sum', 'count']})
song_grouped.columns = ['sum_score', 'count_score']
song_grouped= song_grouped.sort_values(['sum_score', 'count_score', 'songId'],ascending=False)
trending_songs = song_grouped.head(20)
trending_songs #list songId to return by API
#Get Trending Songs: done

,sum_score,count_score
songId,,
sTgsNeblHu8,430,28
icJam_5l,191,10
6o8JoQ8b,162,10
c-6-mB8N,159,12
kd8JSDbB,146,7
zMsUQNJNNoY,144,8
pMhK8ruP,143,9
POhlVMwJ,142,8
rvSmeINN,129,9


In [42]:
top_trending = song.loc[song['songId'].isin(trending_songs.index)]
top_trending.drop_duplicates(['songId'])

,320kbps,album,album_id,artist,duration,genre,has_lyrics,songId,image,language,perma_url,release_date,subtitle,title,url,year
78,NaN,Equals,NaN,Ed Sheeran,214,NaN,NaN,496697,file:///data/user/0/com.hoangminhtai.mobile_pr...,null,NaN,NaN,NaN,I Will Remember You,/storage/emulated/0/CSNMusic/Download/Music/22...,null
139,true,÷,10436096,Ed Sheeran,263,English,false,6o8JoQ8b,https://c.saavncdn.com/286/WMG_190295851286-En...,English,https://www.jiosaavn.com/song/perfect/RgdTexthD1E,2017-03-03,Ed Sheeran - ÷,Perfect,https://aac.saavncdn.com/286/71bb6cc3391ddf619...,2017
238,false,RayeVEVO,UC1aQVIVnCGa4hDJtvNxf11g,RayeVEVO,273,YouTube,false,Dll6VJ2C7wo,https://img.youtube.com/vi/Dll6VJ2C7wo/maxresd...,YouTube,https://www.youtube.com/watch?v=Dll6VJ2C7wo,2022-11-09 00:00:00.000,RayeVEVO,"RAYE, 070 Shake - Escapism. (Official Music Vi...",https://rr1---sn-8qj-nboey.googlevideo.com/vid...,2022
362,false,Evolve,UC0aXrjVxG5pZr99v77wZdPQ,Imagine Dragons,204,YouTube,false,Kx7B-XvmFtE,https://lh3.googleusercontent.com/weYQWfEwWNPO...,YouTube,https://www.youtube.com/watch?v=Kx7B-XvmFtE,2018-07-12 00:00:00.000,Song • Imagine Dragons • Evolve • 3:25,Believer,https://rr6---sn-8qj-nboey.googlevideo.com/vid...,2018
425,true,Dandelions (slowed + reverb),29160455,"Ruth B., sped up + slowed",268,English,false,POhlVMwJ,https://c.saavncdn.com/839/Dandelions-slowed-r...,English,https://www.jiosaavn.com/song/dandelions-slowe...,2021-08-27,"Ruth B., sped up + slowed - Dandelions (slowed...",Dandelions (slowed + reverb),https://aac.saavncdn.com/839/2d3d560559fc503ae...,2021
454,false,FIFTY FIFTY Official,UCJEER74X9kBenMT_x9iK9Mw,FIFTY FIFTY Official,203,YouTube,false,Qc7_zRjH808,https://img.youtube.com/vi/Qc7_zRjH808/maxresd...,YouTube,https://www.youtube.com/watch?v=Qc7_zRjH808,2023-02-23 00:00:00.000,FIFTY FIFTY Official,FIFTY FIFTY (피프티피프티) - 'Cupid' Official MV,https://rr7---sn-42u-nbosr.googlevideo.com/vid...,2023
456,false,Marshmello,UCrxpwXq8wCTskOQq5d_KoqQ,Marshmello,214,YouTube,false,QgKYZWRH4DA,https://img.youtube.com/vi/QgKYZWRH4DA/maxresd...,YouTube,https://www.youtube.com/watch?v=QgKYZWRH4DA,2018-08-09 00:00:00.000,Marshmello - Topic,Happier,https://rr1---sn-8qj-nbo6d.googlevideo.com/vid...,2018
592,false,BLACKPINK,UCOmHUn--16B90oW2L6FRR3A,"Jisoo, BLACKPINK",185,YouTube,false,YudHcBIxlYw,https://img.youtube.com/vi/YudHcBIxlYw/maxresd...,YouTube,https://www.youtube.com/watch?v=YudHcBIxlYw,2023-03-30 00:00:00.000,Jisoo & BLACKPINK,Flower,https://rr2---sn-42u-nbosy.googlevideo.com/vid...,2023
645,true,Flowers,41619705,Miley Cyrus,200,English,false,c-6-mB8N,https://c.saavncdn.com/047/Flowers-English-202...,English,https://www.jiosaavn.com/song/flowers/E0VdHBlyD30,2023-01-13,Miley Cyrus - Flowers,Flowers,https://aac.saavncdn.com/047/985bd26eb8496d9a7...,2023
652,true,Saturday/Sunday,42357833,"Jason Derulo, David Guetta",166,English,true,c-bZOSkM,https://c.saavncdn.com/251/Saturday-Sunday-Eng...,English,https://www.jiosaavn.com/song/saturdaysunday/E...,2023-02-09,"Jason Derulo, David Guetta - Saturday/Sunday",Saturday/Sunday,https://aac.saavncdn.com/251/32868e0a105f2ed2c...,2023


In [1]:
import service as sv
self_sv = sv.firebase_service()
self_sv.get_data_and_preprocessing()
# self_sv.get_trending_songs()

In [3]:
self_sv.get_item_similar_songs(['Chúng Ta Không Thuộc Về Nhau - Sơn Tùng M-TP'])


no. of unique songs in the training set: 741
Non zero values in cooccurence_matrix :463


,user_id,song,score,rank
0,,EM ĐỒNG Ý (I DO) - ĐỨC PHÚC x 911 x KHẮC HƯNG ...,0.80,1
1,,Charlie Puth - Attention [Official Video] - Ch...,0.75,2
2,,Starboy (Kygo Remix) - The Weeknd,0.60,3
3,,Jessi (제시) - '눈누난나 (NUNU NANA)' MV - Jessi,0.60,4
4,,ĐAU ĐỂ TRƯỞNG THÀNH | ONLYC | OFFICIAL MV - On...,0.60,5
5,,SUÝT NỮA THÌ | OFFICIAL OST | CHUYẾN ĐI CỦA TH...,0.60,6
6,,I Will Remember You - Ed Sheeran,0.60,7
7,,The Chainsmokers - Closer (Official Video) ft....,0.60,8
8,,The One That Got Away - Brielle Von Hugel,0.50,9
9,,Phải Chia Tay Thôi (#PCTT) | Tuấn Hưng | Offic...,0.50,10


In [ ]:
self_sv.random_update_stats()

In [2]:
self_sv.get_user_similar_songs('b74d04a0-f5e4-11ed-81a7-85dc82d623d5')

No. of unique songs for the user: 89
no. of unique songs in the training set: 748
Non zero values in cooccurence_matrix :21482


[['b74d04a0-f5e4-11ed-81a7-85dc82d623d5',
  'sTgsNeblHu8',
  0.15015321756894795,
  1],
 ['b74d04a0-f5e4-11ed-81a7-85dc82d623d5', '6o8JoQ8b', 0.12919118958444797, 2],
 ['b74d04a0-f5e4-11ed-81a7-85dc82d623d5',
  'YudHcBIxlYw',
  0.1220572498662386,
  3],
 ['b74d04a0-f5e4-11ed-81a7-85dc82d623d5',
  'oftolPu9qp4',
  0.12078205814160867,
  4],
 ['b74d04a0-f5e4-11ed-81a7-85dc82d623d5', 'icJam_5l', 0.11987248082753697, 5],
 ['b74d04a0-f5e4-11ed-81a7-85dc82d623d5', 'HeooV58S', 0.11987248082753697, 6],
 ['b74d04a0-f5e4-11ed-81a7-85dc82d623d5', '4HOy_-Xm', 0.11639022650258608, 7],
 ['b74d04a0-f5e4-11ed-81a7-85dc82d623d5',
  '0zGcUoRlhmw',
  0.11445068664169784,
  8],
 ['b74d04a0-f5e4-11ed-81a7-85dc82d623d5', '496697', 0.11424112716247543, 9],
 ['b74d04a0-f5e4-11ed-81a7-85dc82d623d5',
  'eoJecvGMR6E',
  0.11301943998573209,
  10],
 ['b74d04a0-f5e4-11ed-81a7-85dc82d623d5',
  '46SGSGti2xQ',
  0.1128321740681291,
  11],
 ['b74d04a0-f5e4-11ed-81a7-85dc82d623d5',
  'Dll6VJ2C7wo',
  0.1125200642054574

In [9]:
self_sv.get_user_similar_songs('0c2983d0-f16d-11ed-9c88-514c38a48ebe')

NameError: name 'self_sv' is not defined